In [1]:
from src.constants.constants import *
from src.feature_extraction.extract_features import *
from src.utils.helper import *
from src.models.evaluate import *
from src.models.train import *
from src.preprocessing.audio_cleaning import *
import warnings

warnings.filterwarnings("ignore")


In [2]:
N_NEIGHBORS

5

In [3]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from src.utils.helper import get_splitted_data


In [4]:
data = pd.read_csv('csv_files/data.csv')
features = pd.read_csv('csv_files/features_40_MFCC.csv')
df = pd.merge(features, 
              data.drop(columns=['client_id', 'sentence', 'up_votes', 'down_votes', 'accent']), 
              left_on='voice_id', right_on='path', how='inner')
df.drop(columns=['path', 'Unnamed: 0'], inplace=True)
df.head()

,mean_mfcc_0,mean_mfcc_1,mean_mfcc_2,mean_mfcc_3,mean_mfcc_4,mean_mfcc_5,mean_mfcc_6,mean_mfcc_7,mean_mfcc_8,mean_mfcc_9,...,std_delta_mfcc_35,std_delta_mfcc_36,std_delta_mfcc_37,std_delta_mfcc_38,std_delta_mfcc_39,pitch,voice_id,age,gender,label
0,-403.22006,99.76226,20.534065,41.544975,-7.227066,31.042414,-2.121207,-0.940516,2.747488,7.995158,...,1.270415,0.916675,1.036257,0.922543,0.850381,1656.88460,common_voice_en_181169.mp3,fifties,male,2
1,-223.81348,157.47336,11.694227,66.153180,-23.514465,0.811838,-19.525723,-8.694965,-6.325938,-10.528963,...,1.447682,0.839711,1.264348,1.171347,1.692225,1285.26820,common_voice_en_122004.mp3,twenties,male,0
2,-415.86557,126.01485,-36.185616,48.158478,-13.574215,-18.600720,-39.063217,-24.592718,-17.863024,-13.136163,...,1.995541,1.724953,1.857788,1.803757,2.830431,1702.50980,common_voice_en_10208833.mp3,fifties,female,3
3,-423.24472,198.54523,12.053857,62.119160,17.754673,-21.133663,6.490955,-1.768311,-28.847279,-15.727254,...,1.094712,2.345557,2.794774,2.051888,1.940512,820.08923,common_voice_en_18618243.mp3,twenties,male,0
4,-223.69318,74.87990,9.623554,114.356290,-12.664084,-20.089724,-38.216280,-0.664027,-42.319390,-13.502878,...,0.912976,1.264993,1.243301,0.944270,1.173495,1860.53880,common_voice_en_18075517.mp3,twenties,male,0


In [5]:
X = df.drop(columns=['label', 'gender', 'age', 'voice_id'])
y = df['label']

X_val_train, X_test, y_val_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_val_train, y_val_train, test_size=0.18, random_state=42, stratify=y_val_train)

In [6]:
Xa_train, Xa_val, ya_train, ya_val = get_splitted_data(X, y, df, 'age')
Xg_train, Xg_val, yg_train, yg_val = get_splitted_data(X, y, df, 'gender')

In [8]:
%%time 
pipeline_a = get_pipeline(name='knn')
model_a = train_model(pipeline_a, Xa_train, ya_train)
model_a

CPU times: user 232 ms, sys: 101 ms, total: 333 ms
Wall time: 344 ms


Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('kneighborsclassifier', KNeighborsClassifier())])

In [9]:
# joblib.dump(model_a, 'src/selected_model/knn_base_age_.pkl')

['src/selected_model/knn_base_age_.pkl']

In [10]:
%%time 
pipeline_g = get_pipeline(name='knn')
model_g = train_model(pipeline_g, Xg_train, yg_train)
model_g

# model_g = load_model("src/saved_models/lgbm_gender_model.pkl")

CPU times: user 228 ms, sys: 96 ms, total: 324 ms
Wall time: 324 ms


Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('kneighborsclassifier', KNeighborsClassifier())])

In [11]:
# joblib.dump(model_g, 'src/selected_model/knn_base_gender_.pkl')

['src/selected_model/knn_base_gender_.pkl']

In [9]:
%%time 
pipeline = get_pipeline(name='knn')
model = train_model(pipeline, X_train, y_train)
model

CPU times: user 242 ms, sys: 109 ms, total: 351 ms
Wall time: 354 ms


Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('kneighborsclassifier',
                 KNeighborsClassifier(metric='cosine'))])

In [10]:
%%time
evaluate_multi(model, X_test, y_test, type = 2)



TESTING RESULTS: 
CONFUSION MATRIX:
[[17495   188   177    95]
 [  126  2601     2   172]
 [  458    20  2104    12]
 [  133   247     7  1987]]
ACCURACY SCORE:
0.9366
CPU times: user 1min 47s, sys: 26.8 s, total: 2min 14s
Wall time: 1min 18s


In [11]:
x = pd.merge(df, X_test, left_index=True, right_index=True, how='right')
x.head()

,mean_mfcc_0_x,mean_mfcc_1_x,mean_mfcc_2_x,mean_mfcc_3_x,mean_mfcc_4_x,mean_mfcc_5_x,mean_mfcc_6_x,mean_mfcc_7_x,mean_mfcc_8_x,mean_mfcc_9_x,...,std_delta_mfcc_31_y,std_delta_mfcc_32_y,std_delta_mfcc_33_y,std_delta_mfcc_34_y,std_delta_mfcc_35_y,std_delta_mfcc_36_y,std_delta_mfcc_37_y,std_delta_mfcc_38_y,std_delta_mfcc_39_y,pitch_y
87069,-281.94406,201.146840,4.391376,68.258840,-17.089682,6.560512,-5.857045,-28.403639,-3.634766,-4.248062,...,1.108591,1.019144,0.846061,1.147466,0.771761,0.787570,1.421859,1.745149,1.901931,669.41296
42665,-308.22190,173.087040,0.801586,-28.245972,-64.956610,-32.546400,-15.978315,-3.659044,-15.762386,-12.324957,...,2.965542,2.513123,2.086046,2.186499,2.532052,2.312073,2.782322,1.886912,1.311566,696.32550
109016,-303.03915,150.930980,-47.686620,53.087147,-26.411861,-18.253426,-18.209720,-20.466350,-14.088178,-14.212386,...,1.290007,1.594329,1.214441,1.636959,2.223634,2.968935,2.589408,3.662922,3.690141,1473.15050
110402,-433.80133,72.147810,7.029837,43.468820,-16.971660,-17.044973,-37.087997,-11.829974,-15.852037,-0.625612,...,1.200773,1.364756,0.936878,1.247755,1.099859,1.076458,0.913637,1.172561,0.927658,1789.12180
80457,-260.18317,109.363174,33.693770,52.015930,14.755984,-24.705837,3.136095,4.209895,-19.941381,6.267535,...,1.040988,1.088045,0.945109,1.317320,1.648822,1.209703,1.940767,2.364728,2.088577,1409.39070


In [12]:
ya_pred = model_a.predict(X_test)
yg_pred = model_g.predict(X_test)

xa = x['age']
lea = LabelEncoder()
xa = lea.fit_transform(xa)
acca_train = accuracy_score(xa, ya_pred)
acca_train

0.9622831474597274

In [13]:
xg = x['gender']
leg = LabelEncoder()
xg = leg.fit_transform(xg)

accg_train = accuracy_score(xg, yg_pred)
accg_train

0.9814126394052045

In [14]:
combined = 3 - (1*yg_pred + 2*ya_pred)
combined

array([0, 3, 0, ..., 0, 2, 1], shape=(25824,))

In [15]:
y_pred = model.predict(X_test)

accl = accuracy_score(combined, y_pred)
accl

0.9631737918215614